In [1]:
!pip install tiktoken

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import tiktoken
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
# Read datasets
train_df = pd.read_csv('/kaggle/input/emotions-dataset-for-nlp/train.txt', names=['text', 'emotion'], sep=';')
validation_df = pd.read_csv('/kaggle/input/emotions-dataset-for-nlp/val.txt', names=['text', 'emotion'], sep=';')
test_df = pd.read_csv('/kaggle/input/emotions-dataset-for-nlp/test.txt', names=['text', 'emotion'], sep=';')

In [4]:
train_df

,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
15995,i just had a very brief time in the beanbag an...,sadness
15996,i am now turning and i feel pathetic that i am...,sadness
15997,i feel strong and good overall,joy
15998,i feel like this was such a rude comment and i...,anger


In [5]:
validation_df

,text,emotion
0,im feeling quite sad and sorry for myself but ...,sadness
1,i feel like i am still looking at a blank canv...,sadness
2,i feel like a faithful servant,love
3,i am just feeling cranky and blue,anger
4,i can have for a treat or if i am feeling festive,joy
...,...,...
1995,im having ssa examination tomorrow in the morn...,sadness
1996,i constantly worry about their fight against n...,joy
1997,i feel its important to share this info for th...,joy
1998,i truly feel that if you are passionate enough...,joy


In [6]:
test_df

,text,emotion
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness
...,...,...
1995,i just keep feeling like someone is being unki...,anger
1996,im feeling a little cranky negative after this...,anger
1997,i feel that i am useful to my people and that ...,joy
1998,im feeling more comfortable with derby i feel ...,joy


In [7]:
from sklearn.preprocessing import LabelEncoder

# Transform labels
le = LabelEncoder()
train_df['label'] = le.fit_transform(train_df['emotion'])
validation_df['label'] = le.transform(validation_df['emotion'])
test_df['label'] = le.transform(test_df['emotion'])

In [8]:
train_df.label.value_counts()

label
2    5362
4    4666
0    2159
1    1937
3    1304
5     572
Name: count, dtype: int64

In [9]:
label_dict = dict(zip(train_df['label'], train_df['emotion']))
label_dict

{4: 'sadness', 0: 'anger', 3: 'love', 5: 'surprise', 1: 'fear', 2: 'joy'}

In [10]:
class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, device):
        self.text = dataframe['text'].str.lower().values
        self.labels = dataframe['label'].values
        self.tokenizer = tokenizer
        self.device = device

    def __len__(self):
        return len(self.text)

    def __getitem__(self, idx):
        text = self.text[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode(text)
        input_ids = torch.tensor(encoding, dtype=torch.long, device=self.device)
        label = torch.tensor(label, dtype=torch.long, device=self.device)
        return input_ids, label

In [11]:
def collate_fn(batch):
    input_ids = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    max_length = max(len(ids) for ids in input_ids)
    
    # All tensors should already be on the correct device from the dataset
    device = input_ids[0].device
    
    input_ids = torch.stack([
        torch.cat([ids, torch.zeros(max_length - len(ids), dtype=torch.long, device=device)]) 
        for ids in input_ids
    ])
    labels = torch.stack(labels)
    return input_ids, labels

In [12]:
import torch
import torch.nn as nn

class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(TransformerEncoderLayer, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.depth = d_model // num_heads

        # Linear layers for Q, K, V matrices
        self.wq = nn.Linear(d_model, d_model)  # (batch_size, seq_len, d_model) -> (batch_size, seq_len, d_model)
        self.wk = nn.Linear(d_model, d_model)  # (batch_size, seq_len, d_model) -> (batch_size, seq_len, d_model)
        self.wv = nn.Linear(d_model, d_model)  # (batch_size, seq_len, d_model) -> (batch_size, seq_len, d_model)

        # Output linear transformation
        self.dense = nn.Linear(d_model, d_model)  # (batch_size, seq_len, d_model) -> (batch_size, seq_len, d_model)   self.

        # Feed-forward network
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, d_ff),  # (batch_size, seq_len, d_model) -> (batch_size, seq_len, d_ff)
            nn.ReLU(),
            nn.Linear(d_ff, d_model)  # (batch_size, seq_len, d_ff) -> (batch_size, seq_len, d_model)
        )

        # Layer normalization and dropout
        self.layernorm1 = nn.LayerNorm(d_model)  # (batch_size, seq_len, d_model) -> (batch_size, seq_len, d_model)
        self.layernorm2 = nn.LayerNorm(d_model)  # (batch_size, seq_len, d_model) -> (batch_size, seq_len, d_model)
        self.dropout = nn.Dropout(dropout)

    def split_heads(self, x, batch_size):
        # Split the last dimension into (num_heads, depth)
        x = x.view(batch_size, -1, self.num_heads, self.depth)  # (batch_size, seq_len, d_model) -> (batch_size, seq_len, num_heads, depth)
        # Transpose the result to shape (batch_size, num_heads, seq_len, depth)
        return x.transpose(1, 2)  # (batch_size, seq_len, num_heads, depth) -> (batch_size, num_heads, seq_len, depth)

    def scaled_dot_product_attention(self, q, k, v, mask=None):
        matmul_qk = torch.matmul(q, k.transpose(-2, -1))  # (batch_size, num_heads, seq_len_q, seq_len_k)
        dk = torch.tensor(k.size(-1), dtype=torch.float32)  # scalar
        scaled_attention_logits = matmul_qk / torch.sqrt(dk)  # (batch_size, num_heads, seq_len_q, seq_len_k)

        if mask is not None:
            scaled_attention_logits = scaled_attention_logits.masked_fill(mask == 0, -1e9)

        attention_weights = torch.nn.functional.softmax(scaled_attention_logits, dim=-1)  # (batch_size, num_heads, seq_len_q, seq_len_k)
        output = torch.matmul(attention_weights, v)  # (batch_size, num_heads, seq_len_q, depth_v)

        return output, attention_weights  # (batch_size, num_heads, seq_len_q, depth_v), (batch_size, num_heads, seq_len_q, seq_len_k)

    def forward(self, x, mask=None):
        batch_size = x.size(0)  # (batch_size, seq_len, d_model)

        # Apply linear layers and split into heads
        q = self.split_heads(self.wq(x), batch_size)  # (batch_size, num_heads, seq_len, depth)
        k = self.split_heads(self.wk(x), batch_size)  # (batch_size, num_heads, seq_len, depth)
        v = self.split_heads(self.wv(x), batch_size)  # (batch_size, num_heads, seq_len, depth)

        # Apply the custom scaled dot-product attention
        scaled_attention, _ = self.scaled_dot_product_attention(q, k, v, mask)  # (batch_size, num_heads, seq_len_q, depth_v)

        # Transpose and reshape back to (batch_size, seq_len, d_model)
        scaled_attention = scaled_attention.transpose(1, 2).contiguous()  # (batch_size, seq_len, num_heads, depth)
        concat_attention = scaled_attention.view(batch_size, -1, self.d_model)  # (batch_size, seq_len, d_model)

        # Apply the final linear layer to combine the heads
        attn_output = self.dense(concat_attention)  # (batch_size, seq_len, d_model)

        # Add & Norm
        x = self.layernorm1(x + self.dropout(attn_output))  # (batch_size, seq_len, d_model)

        # Feed-forward
        ff_output = self.feed_forward(x)  # (batch_size, seq_len, d_model)

        # Add & Norm
        x = self.layernorm2(x + self.dropout(ff_output))  # (batch_size, seq_len, d_model)

        return x  # (batch_size, seq_len, d_model)


![](https://storage.googleapis.com/mle-courses-prod/users/61b6fa1ba83a7e37c8309756/private-files/2dc6f3e0-5fb4-11ef-9b72-9db6eacc12d1-Screen_Shot_2024_08_21_at_18.54.35.png)

In [13]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()

        # Create a long enough 'positional' tensor and fill it with positional encodings
        # [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # (max_len, 1)

        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))  # (d_model/2,)

        # Khởi tạo một tensor `pe` với các giá trị bằng 0, sẽ chứa các giá trị mã hóa vị trí (positional encodings).
        # `pe` có dạng (max_len, d_model), trong đó `max_len` là chiều dài tối đa của chuỗi
        # và `d_model` là chiều kích thước của embeddings của mô hình.
        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)

        # Đối với tất cả các vị trí trong chuỗi (từ 0 đến max_len-1) và đối với tất cả các chỉ số chẵn trong các chiều kích thước embedding,
        # tính toán giá trị sine của tích `position` và `div_term` và gán nó vào các vị trí tương ứng trong `pe`.
        # Gán các giá trị sine cho mọi chiều kích thước khác của embedding (bắt đầu từ chỉ số 0).
        # `position` có dạng (max_len, 1) và `div_term` có dạng (d_model/2,).
        # Phép toán `position * div_term` được broadcast thành dạng (max_len, d_model/2).
        # Kết quả được lưu trữ trong các chỉ số chẵn của `pe`, vì vậy phần được gán có dạng (max_len, d_model/2).
        pe[:, 0::2] = torch.sin(position * div_term)  # (max_len, d_model/2)

        # Tương tự, đối với tất cả các vị trí trong chuỗi và đối với tất cả các chỉ số lẻ trong các chiều kích thước embedding,
        # tính toán giá trị cosine của tích `position` và `div_term` và gán nó vào các vị trí tương ứng trong `pe`.
        # Gán các giá trị cosine cho các chiều kích thước còn lại của embedding (bắt đầu từ chỉ số 1).
        # Giống như trước, `position * div_term` được broadcast thành dạng (max_len, d_model/2).
        # Kết quả được lưu trữ trong các chỉ số lẻ của `pe`, vì vậy phần được gán có dạng (max_len, d_model/2).
        pe[:, 1::2] = torch.cos(position * div_term)  # (max_len, d_model/2)


        # Add a batch dimension and register it as a buffer
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        self.register_buffer('pe', pe)  # Register as buffer so it's not a parameter

    def forward(self, x):
        # x: (batch_size, seq_len, d_model)
        x = x + self.pe[:, :x.size(1), :]  # Add positional encoding, (batch_size, seq_len, d_model)
        return x  # (batch_size, seq_len, d_model)


In [14]:
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embed_size, d_model, num_heads, d_ff, output_size, num_layers, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.positional_encoding = PositionalEncoding(d_model)
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderLayer(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        self.fc = nn.Linear(d_model, output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        x = self.embedding(x)  # (batch_size, seq_len, embed_size)
        x = self.positional_encoding(x)  # (batch_size, seq_len, d_model)
        for layer in self.encoder_layers:
            x = layer(x, mask)  # (batch_size, seq_len, d_model)
        x = x.mean(dim=1)  # (batch_size, d_model)
        x = self.fc(self.dropout(x))  # (batch_size, output_size)
        return x


In [15]:
train_df

,text,emotion,label
0,i didnt feel humiliated,sadness,4
1,i can go from feeling so hopeless to so damned...,sadness,4
2,im grabbing a minute to post i feel greedy wrong,anger,0
3,i am ever feeling nostalgic about the fireplac...,love,3
4,i am feeling grouchy,anger,0
...,...,...,...
15995,i just had a very brief time in the beanbag an...,sadness,4
15996,i am now turning and i feel pathetic that i am...,sadness,4
15997,i feel strong and good overall,joy,2
15998,i feel like this was such a rude comment and i...,anger,0


In [16]:
test_df

,text,emotion,label
0,im feeling rather rotten so im not very ambiti...,sadness,4
1,im updating my blog because i feel shitty,sadness,4
2,i never make her separate from me because i do...,sadness,4
3,i left with my bouquet of red and yellow tulip...,joy,2
4,i was feeling a little vain when i did this one,sadness,4
...,...,...,...
1995,i just keep feeling like someone is being unki...,anger,0
1996,im feeling a little cranky negative after this...,anger,0
1997,i feel that i am useful to my people and that ...,joy,2
1998,im feeling more comfortable with derby i feel ...,joy,2


In [17]:
validation_df

,text,emotion,label
0,im feeling quite sad and sorry for myself but ...,sadness,4
1,i feel like i am still looking at a blank canv...,sadness,4
2,i feel like a faithful servant,love,3
3,i am just feeling cranky and blue,anger,0
4,i can have for a treat or if i am feeling festive,joy,2
...,...,...,...
1995,im having ssa examination tomorrow in the morn...,sadness,4
1996,i constantly worry about their fight against n...,joy,2
1997,i feel its important to share this info for th...,joy,2
1998,i truly feel that if you are passionate enough...,joy,2


In [18]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
# Load and preprocess data

tokenizer = tiktoken.get_encoding('gpt2')

# Create datasets with device
train_dataset = TextDataset(train_df, tokenizer, device)
val_dataset = TextDataset(validation_df, tokenizer, device)
test_dataset = TextDataset(test_df, tokenizer, device)

In [20]:
for input_ids, label in train_dataset:
    print(input_ids)
    print(label)
    break




tensor([   72, 42547,  1254, 42659], device='cuda:0')
tensor(4, device='cuda:0')


In [21]:
# Create dataloaders
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [46]:
# Initialize the Transformer model
vocab_size = tokenizer.n_vocab
embed_size = 256
d_model = 256
num_heads = 8
d_ff = 512
output_size = len(train_df['label'].unique())
num_layers = 3
dropout = 0.2

# Initialize model and move to device
model = TransformerModel(vocab_size, embed_size, d_model, num_heads, d_ff, output_size, num_layers, dropout)
model = model.to(device)

In [47]:
# Initialize loss and optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [24]:
for input_ids, labels in val_dataloader:
    outputs = model(input_ids)
    print(outputs.size())
    print(outputs)
    print()
    print(torch.max(outputs,1))
    print()
    _, predicted = torch.max(outputs,1)
    print(_)
    print(predicted)
    break


torch.Size([32, 6])
tensor([[-0.9681, -0.0581,  1.0815,  0.3145,  0.9513, -0.4801],
        [-0.9470,  0.1412,  1.0625,  0.5648,  1.0482, -0.8358],
        [-0.9632,  0.1790,  0.9773,  0.0340,  1.5458,  0.0882],
        [-1.5710, -0.2870,  1.2744,  0.6619,  1.0289, -0.8508],
        [-0.9681, -0.1050,  1.0676,  0.7444,  1.0113, -0.7773],
        [-0.8513, -0.3751,  0.5126,  0.5425,  1.0086, -0.3616],
        [-1.0313,  0.2228,  0.7346,  0.5097,  0.5218, -0.3493],
        [-1.1895,  0.3365,  1.1550,  0.0657,  0.7774, -0.7578],
        [-1.3988, -0.0792,  0.9848, -0.1299,  1.3961, -0.7063],
        [-0.8254,  0.0139,  1.1990,  0.4045,  1.0936, -0.3269],
        [-0.4513, -0.3821,  0.6357,  0.2269,  1.5763, -0.9907],
        [-1.2924,  0.1783,  0.5985, -0.1882,  0.9726, -0.5302],
        [-0.9497,  0.0864,  0.4326,  0.2306,  1.2132, -0.8298],
        [-1.1875, -0.3001,  1.0294,  0.2445,  1.1197, -0.1908],
        [-0.5547,  0.1921,  0.7546, -0.0142,  1.1118, -0.6957],
        [-0.2526,  0

In [25]:
# Define the accuracy function (using the first version from before)
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy, correct, total

In [26]:
# Training loop with model saving
best_val_accuracy = 0.0  # Track the best validation accuracy
best_model_path = 'best_model.pth'  # File path to save the best model

In [ ]:
num_epochs = 100

for epoch in range(num_epochs):
    # Training step
    model.train()
    train_correct = 0
    train_total = 0
    for input_ids, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(input_ids)
        loss = criterion(outputs, labels)
        
        # Calculate training accuracy
        batch_acc, batch_correct, batch_total = calculate_accuracy(outputs, labels)
        train_correct += batch_correct
        train_total += batch_total
        
        loss.backward()
        optimizer.step()
    
    train_accuracy = 100 * train_correct / train_total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")
    print(f"Correct/Total: {train_correct}/{train_total}, Training Accuracy after Epoch {epoch+1}: {train_accuracy:.2f}%")

    # Validation step
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for input_ids, labels in val_dataloader:
            outputs = model(input_ids)
            batch_acc, batch_correct, batch_total = calculate_accuracy(outputs, labels)
            val_correct += batch_correct
            val_total += batch_total

    val_accuracy = 100 * val_correct / val_total
    print(f"Correct/Total: {val_correct}/{val_total}, Validation Accuracy after Epoch {epoch+1}: {val_accuracy:.2f}%")

    # Save model if validation accuracy improves
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), best_model_path)
    
    print()  # Add a blank line for readability

Epoch 1/100, Loss: 1.8428
Correct/Total: 5736/16000, Training Accuracy after Epoch 1: 35.85%
Correct/Total: 726/2000, Validation Accuracy after Epoch 1: 36.30%

Epoch 2/100, Loss: 1.6083
Correct/Total: 5759/16000, Training Accuracy after Epoch 2: 35.99%
Correct/Total: 731/2000, Validation Accuracy after Epoch 2: 36.55%

Epoch 3/100, Loss: 1.4629
Correct/Total: 5712/16000, Training Accuracy after Epoch 3: 35.70%
Correct/Total: 728/2000, Validation Accuracy after Epoch 3: 36.40%

Epoch 4/100, Loss: 1.4045
Correct/Total: 5737/16000, Training Accuracy after Epoch 4: 35.86%
Correct/Total: 732/2000, Validation Accuracy after Epoch 4: 36.60%

Epoch 5/100, Loss: 1.6317
Correct/Total: 5716/16000, Training Accuracy after Epoch 5: 35.73%
Correct/Total: 727/2000, Validation Accuracy after Epoch 5: 36.35%

Epoch 6/100, Loss: 1.5289
Correct/Total: 5737/16000, Training Accuracy after Epoch 6: 35.86%
Correct/Total: 728/2000, Validation Accuracy after Epoch 6: 36.40%

Epoch 7/100, Loss: 1.6133
Correct/

In [28]:
# Assuming the same model architecture
model.load_state_dict(torch.load('best_model.pth'))
model.eval()  # Set to evaluation mode if using for inference

<ipython-input-28-91a431c573e8>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


TransformerModel(
  (embedding): Embedding(50257, 256)
  (positional_encoding): PositionalEncoding()
  (encoder_layers): ModuleList(
    (0-1): 2 x TransformerEncoderLayer(
      (wq): Linear(in_features=256, out_features=256, bias=True)
      (wk): Linear(in_features=256, out_features=256, bias=True)
      (wv): Linear(in_features=256, out_features=256, bias=True)
      (dense): Linear(in_features=256, out_features=256, bias=True)
      (feed_forward): Sequential(
        (0): Linear(in_features=256, out_features=512, bias=True)
        (1): ReLU()
        (2): Linear(in_features=512, out_features=256, bias=True)
      )
      (layernorm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (layernorm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.3, inplace=False)
    )
  )
  (fc): Linear(in_features=256, out_features=6, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [29]:
test_correct, test_total = 0, 0

for input_ids, labels in test_dataloader:
    outputs = model(input_ids)
    batch_acc, batch_correct, batch_total = calculate_accuracy(outputs, labels)
    test_correct += batch_correct
    test_total += batch_total

test_accuracy = 100 * test_correct / test_total
print(f"Correct/Total: {test_correct}/{test_total}, Accuracy: {test_accuracy:.2f}%")

Correct/Total: 1745/2000, Accuracy: 87.25%


In [30]:
def predict_emotion(sentence, model, tokenizer, device):
    # Preprocess the input sentence
    sentence = sentence.lower()
    encoding = tokenizer.encode(sentence)
    input_ids = torch.tensor(encoding, dtype=torch.long, device=device).unsqueeze(0)
    
    # Make prediction
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids)
        accuracy, predicted = torch.max(outputs, 1)
          
    return label_dict[predicted.item()]

In [31]:
# Example usage
test_sentence = input()
predicted_emotion = predict_emotion(test_sentence, model, tokenizer, device)
print(f"Sentence: {test_sentence}")
print(f"Predicted emotion: {predicted_emotion}")

 àd


Sentence: àd
Predicted emotion: joy
